# MTA Daily Ridership : Maven Analytics Challenge
The daily ridership dataset provides systemwide ridership and traffic estimates for the Metropolitan Transportation Authority's (MTA) different services beginning March 1st, 2020, and provides a percentage comparison against pre-pandemic figures.

In [2]:
from dash import Dash, html, dcc, ctx  # ctx is to know which component we trigger, may be on hover, click 
import dash_ag_grid as dag
from dash.dependencies import Output, Input, State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

import plotly.express as px 
import plotly.graph_objects as go 
import pandas as pd 
import numpy as np 

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
fiv_icons = 'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css'

# Initialize the Dash app
app = Dash(__name__, 
           suppress_callback_exceptions=True,
            meta_tags=[
                {"name": "viewport", "content": "width=device-width, initial-scale=1"}
            ],
           external_stylesheets=[
               dbc.themes.CERULEAN,
               dbc_css,
               fiv_icons
])

load_figure_template("CERULEAN")


# ****************** Data *********************************************************************************************
df = pd.read_csv("MTA_Daily_Ridership.csv", parse_dates=["Date"])

transport_mode_total = df.melt(
    id_vars="Date",
    value_vars=df.columns[1::2],
    var_name="Service Type",
    value_name="Total Volume"
)
transport_mode_total[["Service Type", "Total Estimated"]] = (
    pd.DataFrame(transport_mode_total["Service Type"].str.split(":").to_list())
)
transport_mode_total = transport_mode_total.drop(["Total Estimated"], axis=1)
transport_mode_total = transport_mode_total.sort_values("Date").set_index("Date")

pct_transport_mode = df.melt(
    id_vars="Date",
    value_vars=df.columns[2::2],
    var_name="Service Type",
    value_name="% of Comparable Pre-Pandemic Day"
)
pct_transport_mode[["Service Type", "Total Estimated"]] = (
    pct_transport_mode["Service Type"].str.split(":", expand=True)
)
pct_transport_mode = pct_transport_mode.drop(["Total Estimated"], axis=1)
pct_transport_mode = pct_transport_mode.sort_values("Date").set_index("Date")

mta_data = pd.concat([transport_mode_total, pct_transport_mode.loc[:, "% of Comparable Pre-Pandemic Day"]], axis=1)
mta_data = mta_data.reset_index()
mta_data["Day Name"] =  mta_data["Date"].dt.day_name()
mta_data['Month Name'] = mta_data['Date'].dt.month_name()
mta_data["Weekend"] = np.where(mta_data["Date"].dt.weekday.isin([5, 6]), "Yes", "No")
mta_data["Quarter"] = mta_data["Date"].dt.quarter

mta_data['Season'] = pd.cut(
    mta_data['Date'].dt.month,
    bins=[0, 2, 5, 8, 11, 12],
    labels=['Winter', 'Spring', 'Summer', 'Fall', 'Winter'],
    include_lowest=True,
    ordered=False
)

# Add week number and year
mta_data['Week'] = mta_data['Date'].dt.isocalendar().week
mta_data['Year'] = mta_data['Date'].dt.year

# Utilities (functions) :

# Kpi functions
def calculate_kpis(data):
    
    total_ridership = data['Total Volume'].sum()
    avg_recovery = round(data["% of Comparable Pre-Pandemic Day"].mean(), 1)
    daily_ridership_avg = data["Total Volume"].mean()
    
    service_kpi = ( data.groupby("Service Type")
               .agg({"Total Volume": "sum", "% of Comparable Pre-Pandemic Day": "mean"})
               .to_dict()
              )
    
    return {
        'total_ridership': f"{total_ridership:,.0f}",
        'avg_recovery': f"{avg_recovery}%",
        'daily_ridership_avg': f"{daily_ridership_avg:,.0f}",
        'service_kpi': service_kpi,
       
    }

# Combo chart function

def create_combox_chart(data, title, y1col, y2col, y1aggfunc, y2aggfunc, y1axislabel, y2axislabel, xaxislabel):
    data = data.copy()

    # Aggregating data by year and service type
    data["Year"] = data["Date"].dt.year
    aggregated_data = (
        data.groupby(["Year", "Service Type"], observed=True)
        .agg({
            y1col: y1aggfunc,
            y2col: y2aggfunc,
        })
        .round(2)
        .reset_index()
    )

    # Creating the figure
    fig = go.Figure()

    # Adding the stacked bar chart for y1col
    for service_type in aggregated_data["Service Type"].unique():
        filtered_data = aggregated_data[aggregated_data["Service Type"] == service_type]
        fig.add_bar(
            x=filtered_data["Year"],
            y=filtered_data[y1col],
            name=service_type,
            hovertemplate=( # custom tooltip like Power BI
                'Date: %{x}<br>'  # Show the date
                'Total Ridership: %{y}<br>'  # Total Ridership
                '<extra></extra>'  # Hides the default trace name
            )
        )

    # Adding the line chart for y2col on the secondary y-axis
    yearly_trend = (
        aggregated_data.groupby("Year")
        .agg({y2col: "mean"})
        .reset_index()
    )
    fig.add_scatter(
        x=yearly_trend["Year"],
        y=yearly_trend[y2col],
        mode="lines+markers",
        name="Yearly Recoverate Rate",
        yaxis="y2",
        line={"color": "darkgrey", "width": 2},
        marker={"color": "grey", "size": 5, "symbol": "circle"},
        hovertemplate=(
        'Date: %{x}<br>'  # Show the Year
        'Recovery Rate: %{y:.2f}%<br>'  # Recovery rate
        '<extra></extra>'  # Hides the default trace name
    )
    )

    # Updating the layout for dual y-axes and stacking
    fig.update_layout(
        title=title,
        xaxis_title=xaxislabel,
        yaxis=dict(title=y1axislabel),
        yaxis2=dict(
            title=y2axislabel,
            overlaying="y",
            side="right",
        ),
        barmode="stack",  # Ensures bars are stacked
        legend= {
            "orientation": "h", 
            "xanchor": "center",  
            "x": 0.5,  
            "y": 1.1   
        },
        template="plotly_white",
        height= 600, 

    )

    return fig


# Color map for service types
color_map = {
    'Subways': 'pink',
    'Staten Island Railway': 'blue',
    'Metro-North': 'orange',
    'LIRR': 'purple',
    'Buses': 'green',
    'Bridges and Tunnels': 'red',
    'Access-A-Ride': 'darkblue'
}

# Ridership Distribution by Service Type 
def show_ridership_distribution_service(data):
    data_bar = (data.groupby("Service Type")
               .agg({"Total Volume": "sum"})
               .reset_index()
               .sort_values(by="Total Volume", ascending=False)
    )

    # Calculate percentages
    total = data_bar['Total Volume'].sum()
    data_bar['Percentage'] = data_bar['Total Volume'] / total * 100

    # Create figure with multiple progress bars
    fig = go.Figure()

    # Add a bar for each Service Type
    for idx, row in data_bar.iterrows():
        # Get the color for the service type from the color map
        bar_color = color_map.get(row['Service Type'], 'rgba(200, 200, 200, 0.3)')  # Fallback if not found

        # Add the progress bar with the color from the color map
        fig.add_trace(go.Bar(
            x=[row['Percentage']],
            y=[row['Service Type']],
            orientation='h',
            name=row['Service Type'],
            text=f"{row['Percentage']:.1f}%",
            textposition='auto',
            marker=dict(color=bar_color),  # Main bar color
            showlegend=False
        ))

        # Add a background bar (gray) to show the full 100% with a different color
        fig.add_trace(go.Bar(
            x=[100],
            y=[row['Service Type']],
            orientation='h',
            marker=dict(color='rgba(200, 200, 200, 0.3)'),  # Light gray background
            showlegend=False,
        ))

    # Update layout
    fig.update_layout(
        title={
            'text': "Ridership Distribution by Service Type",
            'y': 0.95,  # Adjust vertical position
            'x': 0.5,   # Center the title horizontally
            'xanchor': 'center',
            'yanchor': 'top'
        },
        barmode='overlay',  # Overlay bars to create progress bar effect
        height=50 * (len(data_bar) + 1),  # Adjust height based on number of services
        margin=dict(l=20, r=20, t=40, b=20),
        xaxis=dict(
            title="Percentage (%)",
            range=[0, 100],
            showgrid=False
        ),
        yaxis=dict(
            title="",
            autorange="reversed"  # Reverse order to match original sorting
        ),
        plot_bgcolor='white'
    )
    
    return fig


# Weekend vs Weekday
def show_ridership_weekend_distribution(data):
    week = data.groupby("Weekend").agg({"Total Volume": "sum"}).reset_index()
    week["Weekend"] = week["Weekend"].map({"No": "Weekday", "Yes": "Weekend"})

    # Custom blue color sequence
    blue_colors = ['#A6C8E1', '#4F89B5']  # Light and dark blue shades

    fig = px.pie(
        week,
        names="Weekend",
        values="Total Volume",
        hole=.7, 
        title="Ridership Distribution: Weekend vs Weekday",
        color="Weekend",
        color_discrete_sequence=blue_colors  # Apply custom color sequence
    )
    
    # Update layout to center the title
    fig.update_layout(
        height = 400,
        title={
            'text': "Ridership Distribution: Weekend vs Weekday",
            'y': 0.95,  # Adjust vertical position
            'x': 0.5,   # Center the title horizontally
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )
    
    return fig


# Heatmap
def create_heatmap(data, value_col, title):
    pivot_data = data.pivot_table(
        index='Day Name',
        columns='Month Name',
        values=value_col,
        aggfunc='mean',
        observed=True,  
    )
    
    # Reorder days of week
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    pivot_data = pivot_data.reindex(days_order)
    
    fig = px.imshow(
        pivot_data,
        color_continuous_scale='Plotly3',  # Prebuilt blue color scale
        aspect='auto',
    )
    fig.update_layout(
        margin=dict(l=20, r=20, t=40, b=20),
        title={
            'text': title,
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
        },
        font=dict(
            family="Arial, sans-serif",
            size=14,
        )
    )
    fig.update_xaxes(title_text="Month", showgrid=False)
    fig.update_yaxes(title_text="Day", showgrid=False)
    return fig







# Define navigation items, like that to shorten the code
nav_items = {
    '/': {'title': 'Overview', 'icon': 'fas fa-home', 'description': 'Main dashboard view'},
    '/service-level': {'title': 'Service Type', 'icon': 'fas fa-headset', 'description': 'Monitor service metrics'}
}

# Contact details

my_contact_details = html.Div(
    dbc.Container(
            dbc.Row(
                [
                    dbc.Col(
                        html.Div(
                            [
                                html.Img(
                                    src='assets/profile.jpeg',  
                                    style={'width': '100px', 'border-radius': '50%'}
                                ),
                                html.H4("TIGA SAWADOGO"),
                                html.H4("Contact Me"),
                                html.A(
                                    "tiga.sawadogo@centrale-casablanca.ma",
                                    href="mailto:tiga.sawadogo@centrale-casablanca.ma", 
                                    target="_blank",
                                    style={'text-decoration': 'none', 'color': 'blue'}
                                ),
                                html.Br(),
                                
                                html.A(
                                    "LinkedIn",
                                    href="https://www.linkedin.com/in/tiga-sawadogo-8b65ab19a/", 
                                    target="_blank",
                                    style={'text-decoration': 'none', 'color': 'blue'}
                                ),
                            ],
                            style={'text-align': 'center', 'padding': '10px'}
                        ),
                        width=12
                    )
                ]
            ),
            fluid=True
        ),
        style={
            'bottom': '0',
            'left': '0',
            'width': '100%',
            'height': "20%",
            'border-top': '1px solid #ccc',
            'box-shadow': '0 -2px 5px rgba(0,0,0,0.1)',
            'text-align': 'center'
        }
)
 

service_types = { 
     "subways": {
        "name": "Subways",
    },
    "access-a-ride": {
        "name": "Access-A-Ride",
    },
    "metro-north": {
        
        "name": "Metro-North",
    },
    "buses": {
        "name": "Buses",
    },
    "staten-island-railway": {
        "name": "Staten Island Railway",
    },
    "bridges-and-tunnels":{
      
        "name": "Bridges and Tunnels",
    },
    "lirr" : {
        "name": "LIRR",
    }
}

card_style = {
    'borderRadius': '8px',
    'boxShadow': '0 2px 6px rgba(0,0,0,0.15)',
    'padding': '15px',
    'margin': '10px',
    "heigh": "50%"
}


kpi_card_style = {
    'backgroundColor': 'rgba(240, 248, 255, 0.9)',  # Light blue background, consistent with CERULEAN
    'borderRadius': '25px',
    'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.2)',
    'padding': '10px',
    'margin': '5px',
    'height': '100%',
    'color': '#003366',  
}


# Create Navbar
navbar = dbc.Row(
    [
        # Logo Column
        dbc.Col(
            html.Img(
                src="assets/mta-blue-logo-rectangle.png",  # Replace with your logo's path
                style={"height": "50px"}
            ),
            width="auto"
        ),

        # Title Column
        dbc.Col(
            html.H1("MTA Ridership Dashboard", className="text-center my-4"),
            style={"display": "flex", "align-items": "center", "color": "darkblue", "fontSize" : "30px"}
        ),

        # Navigation items
        dbc.Col(
            dbc.Nav(
                [
                    dbc.NavItem(
                        dbc.NavLink(
                            [
                                html.I(className=info['icon'], style={'margin-right': '10px'}),
                                info['title']
                            ],
                            href=path,
                            className="nav-link",
                            style = {}, # This will be dynically updated
                            id=f"nav-{path[1:] or 'home'}"
                        )
                    ) for path, info in nav_items.items()
                ],
                horizontal=True,
                class_name="ms-3"
            ),
            width=True
        )
    ],
    className="border-bottom pb-2 pt-2 mb-4 bg-white",
    align="center"
)

# ***********************************************************************************************************************
# Define layouts for different pages
overview_layout = html.Div([
    dbc.Row([
        # DateRangePicker Column
        dbc.Col([
            dcc.DatePickerRange(
                id='date-picker',
                min_date_allowed=mta_data['Date'].min(),
                max_date_allowed=mta_data['Date'].max(),
                start_date=mta_data['Date'].min(),
                end_date=mta_data['Date'].max(),
                display_format='MMM DD, YYYY',
                style={"width": "100%", "font-weight": "bold"}
            ),
            
        ], width=3,), 
        # KPI Row
        dbc.Col([
          
                dbc.Row([
                dbc.Col([
                    dbc.Card([
                        html.H4("Total Ridership", className="text-muted"),
                        html.H2(id="total-ridership-kpi", className="text-primary")
                    ], style=kpi_card_style)
                ], width=4),
                
                dbc.Col([
                    dbc.Card([
                        html.H4("Average Recovery", className="text-muted"),
                        html.H2(id="avg-recovery-kpi", className="text-success")
                    ], style=kpi_card_style)
                ], width=4),
                
                dbc.Col([
                    dbc.Card([
                        html.H4("Daily Average", className="text-muted"),
                        html.H2(id="daily-avg-kpi", className="text-info")
                    ], style=kpi_card_style)
                ], width=4)
            ], className="mb-4"),
            
        ], width=8)
    ]),
    
    html.Br(),
    
    # Graphics
    dbc.Row([
           dbc.Col([
            dbc.Card([dcc.Graph(id= "combo-ridership-recovery")], style=card_style)
        ], width=12),
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dcc.Graph(id= "ridership_distribution_service")
            ], style= card_style)], width=6),
        dbc.Col([
            dbc.Card([
                dcc.Graph(id="ridership_weekend_distribution")
            ],  style=card_style)
           ], width=6),     
            
    ]),
    html.Br(),
    dbc.Row([
           dbc.Col([
           dbc.Card([dcc.Graph(id= "ridership-heatmap", clear_on_unhover=True)],  style=card_style)
        ], width=6),
           dbc.Col([
            dbc.Card([dcc.Graph(id= "recovery-heatmap", clear_on_unhover=True)],  style=card_style)
           ], width=6),
       ]),
    html.Br(),
    dbc.Card([
            dbc.Row([
                dbc.Col(id="heatmap-on-hover-table-AgGrid", width=6),
                dbc.Col(dcc.Graph(id="heatmap-on-hover-bar-plot"), width=6),
            ])
        ], style=card_style),
    
    dcc.Store(id="filter_date_range_data", storage_type='session'), # store the data in browser for use
    
])


# *********************************************************************************************************************
service_level_layout = html.Div([
    html.H1("Service Type"),
    html.P("Get Insights on different service types (transport mode)."),
    dcc.DatePickerRange(
        id='date-picker-service',
        min_date_allowed=mta_data['Date'].min(),
        max_date_allowed=mta_data['Date'].max(),
        start_date=mta_data['Date'].min(),
        end_date=mta_data['Date'].max(),
        display_format='MMM DD, YYYY',
        style={"width": "100%", "font-weight": "bold"}
    ),
    html.Br(),
    html.Br(),
    dbc.Row([ 
            dbc.Col([
                dbc.Button(service_types[key]['name'], 
                           id=f"btn-{key}", 
                           n_clicks=0, 
                            style={
                                "fontSize": 16, 
                                "width": "100%",  # each button to expand and fill the available space within the column
                                "height": "50px",  
                                "marginRight": "10px",
                            }
                          ),
                dbc.Offcanvas(
                    [
                        html.H4(id=f"text-{key}"),
                        html.Hr(),
                        html.H5("Select Time Period:", 
                               className="mb-3",  # Add margin bottom
                               style={"color": "#2C3E50"}  # Dark blue-gray color
                        ),
                        dcc.Checklist(
                            id= f"time-component-{key}",
                            options = [
                                {"label" : "Day", "value": "Date"},
                                {"label": "Week", "value": "Week"},
                                {"label": "Month", "value": "Month Name"}, 
                                {"label": "Year", "value": "Year"},
                                {"label": "Season", "value": "Season"}
                            ],
                            value=[],
                            style={
                                'gap': '25px',
                                'padding': '20px',
                                'backgroundColor': '#f8f9fa',
                                'borderRadius': '8px',
                                'border': '1px solid #e9ecef',
                            }
                        ),
                        html.Hr(),
                        html.H5("Select Metric(s):", 
                               className="mb-3",  # Add margin bottom
                               style={"color": "#2C3E50"}  # Dark blue-gray color
                        ),
                        
                       dcc.Checklist(
                        id=f"metric-{key}", 
                        options = [
                            {"label": "Ridership Volume", "value": "Total Volume"},
                            {"label":"Recovery Rate", "value": "% of Comparable Pre-Pandemic Day"}
                        ],
                        value=[], 
                        style={
                            'gap': '25px',
                            'padding': '20px',
                            'backgroundColor': '#f8f9fa',
                            'borderRadius': '8px',
                            'border': '1px solid #e9ecef',
                        }
                        ),
                    ],

                    id=f"offcanvas-{key}",
                    is_open=False,
                ),
            ],  
            ) for key in service_types.keys()
        ], ),

    html.Br(), 
    html.Div(id='graphs-container', children=[], style={"width": "100%", "height": "100%"}), # we will add dynamically graphs in this container
    dcc.Store(id='calendar_filtered_data-store'), # Store data filtered from the calendar
    dcc.Store(id='offcanvas-store-data', data={}), # we will store the filtered data in the browser
    dcc.Store(id='store-time-periods', data={}),  # Store for time periods
    dcc.Store(id='store-metrics', data={}),# Store for metrics
    
    dcc.Store(id='previous-state-store', data={}) 
    # allows the app to compare the current and previous states to determine whether to clear the graphs

    
   
])

# *********************************************************************************************************************
# App layout
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        dbc.Row(
            [
                html.Div(navbar, className="mb-4"), # Navigation bar
            
            ],
            justify="between",  # Space out the items
            align="center",  # Align items vertically
            className="mb-4"
        ),
        
        html.Div(id='page-content')  # the content of each page
    ], style={'padding': '20px'}),
    
    my_contact_details,
])

# Callback for page routing
@app.callback(
    Output('page-content', 'children'),
    Input('url', 'pathname')
)
def display_page(pathname):
    
    if pathname == '/service-level':
        return service_level_layout
    else:
        return overview_layout
    
# Callback to update active nav link style
@app.callback(
    [Output(f"nav-{path[1:] or 'home'}", "style") for path in nav_items.keys()],
    [Input('url', 'pathname')]
)
def update_nav_style(pathname):
    base_style = {
        'color': '#4b5563',
        'font-size': '25px',
        'padding': '8px 16px'
    }
    active_style = {
        **base_style,
        'color': '#3b82f6'
    }
    
    return [active_style if path == pathname else base_style for path in nav_items.keys()]


# Overview page content callback functions

@app.callback(
    [
        Output(component_id="total-ridership-kpi", component_property="children"),
        Output(component_id="avg-recovery-kpi", component_property="children"),
        Output(component_id="daily-avg-kpi", component_property="children"),
        
        Output(component_id="combo-ridership-recovery", component_property="figure"),
        Output(component_id="ridership_distribution_service", component_property="figure"),
        Output(component_id="ridership_weekend_distribution", component_property="figure"),
        
        Output(component_id="ridership-heatmap", component_property="figure"),
        Output(component_id="recovery-heatmap", component_property="figure"),
        
        Output(component_id="filter_date_range_data", component_property="data") # to retrieve data from the callback func
    ],
    [
        Input(component_id="date-picker", component_property="start_date"),
        Input(component_id="date-picker", component_property="end_date"),
       
    ]
)

def update_dashboard(start_date, end_date):
    # Filter data based on selections
    mask = (
        (mta_data['Date'] >= start_date) &
        (mta_data['Date'] <= end_date)
    )
    filtered_data = mta_data[mask]
    
    kpis = calculate_kpis(filtered_data)
    
    combo_chart_ridership_recovery_rate = create_combox_chart(data = mta_data, 
                    title="Total Ridership & Average Recovery Rate by Year", 
                    y1col="Total Volume", 
                    y1aggfunc= "sum",
                    y2col="% of Comparable Pre-Pandemic Day",
                    y2aggfunc = "mean",
                    y1axislabel= "Total Ridership",
                    y2axislabel= "Recoverate rate",
                    xaxislabel= "Year"
                   )
    
    ridership_distribution_service = show_ridership_distribution_service(filtered_data)
    
    ridership_weekend_distribution =  show_ridership_weekend_distribution(filtered_data)
    
    
    ridership_heatmap = create_heatmap(
        filtered_data,
        'Total Volume',
        'Ridership Heatmap by Day and Month',
    )
    
    recovery_heatmap = create_heatmap(
        filtered_data,
        '% of Comparable Pre-Pandemic Day',
        'Recovery Rate Heatmap by Day and Month'
    )
    
#     print(len(filtered_data.to_dict("records")))
    
    return (
        kpis["total_ridership"],
        kpis["avg_recovery"],
        kpis["daily_ridership_avg"],
        combo_chart_ridership_recovery_rate,
        ridership_distribution_service,
        ridership_weekend_distribution,
        ridership_heatmap,
        recovery_heatmap,
        filtered_data.to_dict("records") # This data will be store in browser user session, and will use when hover heatmaps
    )

@app.callback(
    [
        Output(component_id="heatmap-on-hover-table-AgGrid", component_property="children"),
        Output(component_id="heatmap-on-hover-bar-plot", component_property="figure"),
    ],  
    [
        Input(component_id="filter_date_range_data", component_property="data"),
        Input(component_id="ridership-heatmap", component_property="hoverData"),
        Input(component_id="recovery-heatmap", component_property="hoverData"),
    ]
)

def update_hover_summary(data, hoverData1, hoverData2):
    filtered_data = pd.DataFrame(data) # Transform json to dataframe.
    filtered_data["Date"] = pd.to_datetime(filtered_data['Date']) 
   
    heatmap_is_hover = ctx.triggered_id # this get the id of component (heatmap) which I hover,trigger
    
    attribute = "Ridership"
    
    if heatmap_is_hover and hoverData1:
        x_val = hoverData1['points'][0]['x']
        y_val = hoverData1['points'][0]['y']
        z_val = round(hoverData1['points'][0]['z'], 0) # Total Ridership
        
        that_day_month= (filtered_data.loc[(filtered_data["Month Name"] == f'{x_val}')
                                      & (filtered_data["Day Name"] == f'{y_val}')]
                   )
        
        # Prepare bar plot data for Ridership by Year
        bar_data = (that_day_month.groupby(that_day_month["Date"].dt.year)
                    .agg({"Total Volume": "sum"})
                    .sort_values("Total Volume")).reset_index()
        
        bar_data = bar_data.rename({"Date": "Year"}, axis=1)
        
        
        bar_plot = px.bar(
            bar_data,
            y= "Year",
            x="Total Volume",
            orientation='h'
        )
        
        # Update the layout to center the title
        bar_plot.update_layout(
            title={
                'text': f"Ridership Volume by Year in {x_val} on {y_val}",
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            }
        )

        # Center the X-axis label
        bar_plot.update_xaxes(
            title={
                'text': "Ridership Volume",
                'standoff': 20 
            },
            title_standoff=20  
        )
    
    elif heatmap_is_hover and hoverData2:
        attribute = "Recovery Rate"
        x_val = hoverData2['points'][0]['x']
        y_val = hoverData2['points'][0]['y']
        z_val = f'{round(hoverData2['points'][0]['z'], 1)}%'
        
        that_day_month = (filtered_data.loc[(filtered_data["Month Name"] == f'{x_val}')
                                      & (filtered_data["Day Name"] == f'{y_val}')]
                   )
        
        # Prepare bar plot data for Ridership by Year
        bar_data = (that_day_month.groupby(that_day_month["Date"].dt.year)
                    .agg({"% of Comparable Pre-Pandemic Day": "mean"})
                    .sort_values("% of Comparable Pre-Pandemic Day")).reset_index()
        
        bar_data =  bar_data.rename(columns={"Date": "Year"})
        
        bar_plot = px.bar(
            bar_data,
            y= "Year",
            x="% of Comparable Pre-Pandemic Day",
            orientation='h'
        )
        
        
        # Update the layout to center the title
        bar_plot.update_layout(
            title={
                'text': f"Ridership Recovery Rate by Year in {x_val} on {y_val}",
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            }
        )

        # Center the X-axis label
        bar_plot.update_xaxes(
            title={
                'text': "Recovery Rate",
                'standoff': 20 
            },
            title_standoff=20  
        )
        
    else : 
        attribute = "Ridership"
        x_val =  "October" 
        y_val = "Wednesday"
        z_val = 880857
        that_day_month = (filtered_data.loc[(filtered_data["Month Name"] == f'{x_val}')
                                      & (filtered_data["Day Name"] == f'{y_val}')]
                   )
        
        # Prepare bar plot data for Ridership by Year
        bar_data = (that_day_month.groupby(that_day_month["Date"].dt.year)
                    .agg({"Total Volume": "sum"})
                    .sort_values("Total Volume")).reset_index()
        
        bar_data =  bar_data.rename(columns={"Date": "Year"})
        
        bar_plot = px.bar(
        bar_data,
        y="Year",
        x="Total Volume",
        orientation='h',
    )

        # Update the layout to center the title
        bar_plot.update_layout(
            title={
                'text': f"Ridership Volume by Year in {x_val} on {y_val}",
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            }
        )

        # Center the X-axis label
        bar_plot.update_xaxes(
            title={
                'text': "Ridership Volume",
                'standoff': 20 
            },
            title_standoff=20  
        )

        
    
    # Prepare the summary table for AgGrid Table
    table_data = (that_day_month.groupby("Service Type")
                  .agg({"Total Volume": "sum", "% of Comparable Pre-Pandemic Day": "mean"})
                  .sort_values(by="Total Volume", ascending=False)
                  .reset_index()
                  .assign(
                      pct_volume=lambda x: x["Total Volume"] / x["Total Volume"].sum())
                  .reindex(columns=["Service Type","Total Volume", "pct_volume", "% of Comparable Pre-Pandemic Day"])
                 )
    
    # Table visual 
    
    columnDefs = [
        {
            "field": "Service Type", 
            "filter": "agTextColumnFilter",
            "headerClass": "header-bold"
        },
        {
            "field": "Total Volume",
            "headerName": "Total Ridership",
            "filter": "agNumberColumnFilter",
            "valueFormatter": {"function": "d3.format(',')(params.value)"},
            "type": "numericColumn",
            "headerClass": "header-bold"
        },
        {
            "field": "pct_volume",
            "headerName": "% of Total Volume",
            "filter": "agNumberColumnFilter",
            "valueFormatter": {"function": "(params.value * 100).toFixed(2) + '%'"},
            "type": "numericColumn",
            "headerClass": "header-bold"
        },
        {
            "field": "% of Comparable Pre-Pandemic Day",
            "headerName": "Recoverate Rate",
            "filter": "agNumberColumnFilter",
            "valueFormatter": {"function": "params.value.toFixed(2) + '%'"},
            "type": "numericColumn",
            "headerClass": "header-bold"
        }
    ]

     # Table visual  : Using AgGrid
    table_view = dbc.Col([
        html.P(f"In {x_val} {y_val} all years, we have {z_val} {attribute}",
                style={
                        "textAlign": "center",  
                        "color": "darkslategray",  
                        "margin": "10px 0"
                    }
        ),

        dag.AgGrid(
        columnDefs=columnDefs,
        rowData=table_data.to_dict('records'),
        defaultColDef={
            "resizable": True,
            "sortable": True,
            "filter": True,
            "minWidth": 150
        },
        dashGridOptions={
            "domLayout": "autoHeight",
            "rowClass": "grid-row",
            "pagination": True,
            "paginationPageSize": 7
        },
        className="ag-theme-material",
        style={
        "height": "100%", 
        "width": "100%",
        'backgroundColor': 'rgba(240, 248, 255, 0.9)',  # Light blue background to match theme
        'color': '#003366'  # Dark blue text for consistency
        }
        )
    ])
    
    
    return table_view, bar_plot

# services insights

# Callback for date filtering from calendar selection
@app.callback(
    Output('calendar_filtered_data-store', 'data'),
    [Input('date-picker-service', 'start_date'),
     Input('date-picker-service', 'end_date')]
)
def filter_data(start_date, end_date):
    if not start_date or not end_date:
        raise PreventUpdate
    
    service_df = mta_data[
        (mta_data['Date'] >= pd.to_datetime(start_date)) &
        (mta_data['Date'] <= pd.to_datetime(end_date))
    ]
    return service_df.to_dict('records')



# Callback for Offcanvas toggling
@app.callback(
    [Output(f"offcanvas-{key}", "is_open") for key in service_types.keys()],
    [Input(f"btn-{key}", "n_clicks") for key in service_types.keys()],
    [State(f"offcanvas-{key}", "is_open") for key in service_types.keys()]
)
def toggle_offcanvas(*args):
    n_clicks = args[:len(service_types)] # since I am usig for loop in input
    is_open = args[len(service_types):]
    
    if not ctx.triggered_id:
        return [False] * len(service_types)
    
    button_key = ctx.triggered_id.replace("btn-", "")
    button_idx = list(service_types.keys()).index(button_key)
    
    new_states = [False] * len(service_types)
    new_states[button_idx] = not is_open[button_idx]
    
    return new_states


@app.callback(
    [Output(component_id=f"text-{key}", component_property="children") for key in service_types.keys()],
    [Input(component_id=f"btn-{key}", component_property="n_clicks") for key in service_types.keys()],
)
def display_text(*args):
    if not ctx.triggered_id:
        raise PreventUpdate
    
    button_key = ctx.triggered_id.replace("btn-", "")
    
    # Create a list of text values for each service type
    text_values = []
    for key in service_types.keys():
        if key == button_key:
            text_values.append(f"Getting insights from {service_types[key]['name']}")
        else:
            text_values.append("")  # Empty string for other service types

    return text_values



# Store selected service type and its selections
@app.callback(
    Output('offcanvas-store-data', 'data'),
    [Input(f"btn-{key}", "n_clicks") for key in service_types.keys()],
)
def store_service_type(*args):
    if not ctx.triggered_id:
        raise PreventUpdate
    
    button_key = ctx.triggered_id.replace("btn-", "")
    return {
        "offcanvas_id": f"offcanvas-{button_key}",
        "service_type": service_types[button_key]["name"]
    }

# Combined callback for both metrics and time periods
@app.callback(
    [Output('store-time-periods', 'data'),
     Output('store-metrics', 'data')],
    [Input(f"time-component-{key}", "value") for key in service_types.keys()] +
    [Input(f"metric-{key}", "value") for key in service_types.keys()],
    Input('offcanvas-store-data', 'data')
)
def store_selections(*args):
    num_services = len(service_types)
    time_periods = args[:num_services]
    metrics = args[num_services:-1]  # -1 to exclude offcanvas_data
    offcanvas_data = args[-1]
    
    if not offcanvas_data:
        return {}, {}
        
    # Get current service type
    current_service = offcanvas_data.get('service_type')
    
    # Process time periods
    all_periods = set()
    for i, periods in enumerate(time_periods):
        if periods:
            all_periods.update(periods)
            
    # Process metrics
    all_metrics = set()
    for i, metric_list in enumerate(metrics):
        if metric_list:
            all_metrics.update(metric_list)
    
    return (
        {'time_periods': list(all_periods)},
        {'metrics': list(all_metrics)}
    )

# And modify your reset callback to:
@app.callback(
    [Output(f"time-component-{key}", "value") for key in service_types.keys()] +
    [Output(f"metric-{key}", "value") for key in service_types.keys()],
    Input('offcanvas-store-data', 'data')
)
def reset_selections(offcanvas_data):
    if not ctx.triggered:
        raise PreventUpdate
        
    time_outputs = []
    metric_outputs = []
    
    for key in service_types.keys():
        time_outputs.append([])
        metric_outputs.append([])
        
    return time_outputs + metric_outputs



# Fonction for graph creation (outside the callback)
def create_time_series_plot(data, period, metric, service_name):
    
    df_grouped = None # Default value for df_grouped
    
    if period == "Date":
        df_grouped = data
    if period == "Week":
        df_grouped =  data.groupby("Week").agg({"Total Volume": "sum", "% of Comparable Pre-Pandemic Day": "mean"}).reset_index()
    if period == "Month Name": # for Month
        data["Month Number"] = data["Date"].dt.month 
        df_grouped =  (data
                       .groupby(["Month Name", "Month Number"])
                       .agg({"Total Volume": "sum", "% of Comparable Pre-Pandemic Day": "mean"})
                       .reset_index()
                       .sort_values(by="Month Number")
                       .drop(columns={"Month Number"})
                      )        
       
    if period == "Year":
        df_grouped =  data.groupby("Year").agg({"Total Volume": "sum", "% of Comparable Pre-Pandemic Day": "mean"}).reset_index()
    if period == "Season":
        df_grouped =  data.groupby("Season").agg({"Total Volume": "sum", "% of Comparable Pre-Pandemic Day": "mean"}).reset_index()
    
     # Check if df_grouped is assigned
    if df_grouped is None:
        raise ValueError("The period does not match any of the expected values.") 
    
    if period == "Year" or period == "Season":
        fig =  px.bar(df_grouped, 
                  x=period, 
                  y=metric, 
#                   color='Service Type',
                  title=f'{"Total Ridership" if metric == "Total Volume" else "Recovery Rate"} by {"Day" if period == "Date" else period} for {service_name}')
    else:
        fig = px.line(df_grouped, 
                  x=period, 
                  y=metric, 
#                   color='Service Type',
                  title=f'{"Total Ridership" if metric == "Total Volume" else "Recovery Rate"} by {"Day" if period =="Date" else period} for {service_name}')
    
    fig.update_layout(
        xaxis_title= "Day" if period == "Date" else period,
        yaxis_title= "Ridership" if metric == "Total Volume" else "Recovery Rate",
        height=400
    )
    
    return fig

        
        
        
# Callback for graph generation
@app.callback(
    [
        Output('graphs-container', 'children'),
        Output('previous-state-store', 'data'), # we store the previous state (btn click, checkboxs state)
        
    ],
    [
        Input('calendar_filtered_data-store', 'data'),
        Input('offcanvas-store-data', 'data'),
        Input('store-time-periods', 'data'),
        Input('store-metrics', 'data'),
        State('previous-state-store', 'data')
    ]
    
)
def insert_graphs(calendar_filtered_data, offcanvas_data, periods, metrics, prev_state):
     # Default message for graph container
    default_message = [html.Div(
        "Select service type, time periods, and metrics you want to display graphs dynamically.",
        className="text-center p-4")]
    
    # Check if inputs are missing
    if not all([calendar_filtered_data, offcanvas_data, periods, metrics]):
        return (default_message, prev_state)
    
    df = pd.DataFrame(calendar_filtered_data) #  convert json data of dcc.Store to pandas dataframe
    df['Date'] = pd.to_datetime(df['Date'])
    
        
     # Filter data by selected service type
    selected_service_type = offcanvas_data.get('service_type')
    if selected_service_type:
        df = df.loc[df['Service Type'] == selected_service_type]

    # Check if state has changed
    current_state = {
        'service_type': selected_service_type,
        'start_date': df['Date'].min() if not df.empty else None,
        'end_date': df['Date'].max() if not df.empty else None,
        'time_periods': periods.get('time_periods', []),
        'metrics': metrics.get('metrics', [])
    }
     
   # Check if a different button is clicked (service type has changed)
    if prev_state and current_state['service_type'] != prev_state.get('service_type'):
        # Reset time periods and metrics stored values if the service type has changed
        return (default_message, current_state)
    
    # Check if no changes in state
    if prev_state == current_state:
        raise PreventUpdate
        
    graphs = []     
            
    for metric in metrics.get('metrics', []):
        for period in periods.get('time_periods', []):
            graphs.extend([  # extend , python list operation to add each element
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(
                            figure=create_time_series_plot(df, period, metric, selected_service_type),
                            style={"width": "100%"}
                        )
                    ], width=12)
                ], className='mb-4')
            ])
            

    return (graphs if graphs else default_message, current_state)


        

# Run the server
if __name__ == "__main__":
    app.run_server(debug=True, port=2012)

In [63]:
mta_data

,Date,Service Type,Total Volume,% of Comparable Pre-Pandemic Day,Day Name,Month Name,Weekend,Quarter,Season,Week,Year
0,2020-03-01,Subways,2212965,97,Sunday,March,Yes,1,Spring,9,2020
1,2020-03-01,Access-A-Ride,19922,113,Sunday,March,Yes,1,Spring,9,2020
2,2020-03-01,Metro-North,55825,59,Sunday,March,Yes,1,Spring,9,2020
3,2020-03-01,Buses,984908,99,Sunday,March,Yes,1,Spring,9,2020
4,2020-03-01,Staten Island Railway,1636,52,Sunday,March,Yes,1,Spring,9,2020
...,...,...,...,...,...,...,...,...,...,...,...
11937,2024-10-31,LIRR,257756,82,Thursday,October,No,4,Fall,44,2024
11938,2024-10-31,Access-A-Ride,37639,126,Thursday,October,No,4,Fall,44,2024
11939,2024-10-31,Buses,1297891,58,Thursday,October,No,4,Fall,44,2024
11940,2024-10-31,Metro-North,221856,76,Thursday,October,No,4,Fall,44,2024
